# 游戏设置

- 打开游戏 《只狼：影逝二度》
    - 设定
        - 显示和声音
            - 亮度调整
                - 设置 5
        - 按键设置
            - 移动
                - 移动 前
                    - 设置 W
                - 移动 后
                    - 设置 S
                - 移动 左
                    - 设置 A
                - 移动 右
                    - 设置 D
                - 垫步、（长按）冲刺
                    - 设置 .Shift
                - 跳跃
                    - 设置 Space
            - 视角操作
                - 重置视角/固定目标
                    - 设置 Y
            - 攻击动作
                - 攻击
                    - 设置 J
                - 防御
                    - 设置 K
        - 图像设定
            - 屏幕模式
                - 窗口
            - 屏幕分辨率
                - 1280 x 720
            - 自动绘图调整
                - OFF
            - 质量设定
                - 低

## 校准游戏窗口位置

- 将游戏窗口放在左上方  
![example_01](./imgs/example_01.png) 

运行以下代码，然后根据弹出来的窗口调整游戏窗口的位置

In [ ]:
from pysekiro.adjustment import main

main()

![adjustment](./imgs/adjustment.jpg) 

# 开始

## 用深度学习从已有的数据中学习

### 收集数据

1. 建议使用《只狼：影逝二度》v1.06版本
2. 建议通过鬼佛处“再战稀世强者”收集数据
3. 建议在战斗开始时开始搜集数据，战斗结束时停止，保证每一个数据集都是一场完整的战斗。

- 以下是收集“苇名弦一郎”战斗数据的示例。
- 如果想收集其他Boss的战斗数据，那就把target改成对应的Boss的名字（建议使用英文）

In [ ]:
from pysekiro.collect_data import Data_collection
target = 'Genichiro_Ashina' # 苇名弦一郎
c = Data_collection(target)
c.collect_data()

- 当提示 Ready! 时，第一次按 T 开始搜集数据
- 之后按 T 切换暂停和继续的状态
- 按 P 结束

### 训练模型

- 以下是用“苇名弦一郎”战斗数据来训练的示例
    - **target** 指定训练的对象
    - 注：必须 start < end
        - **start** 指定第一个训练的数据集
        - **end** 指定最后一个训练的数据集
    - **batch_size** 训练时的批大小
    - **epochs** 每个数据集训练的次数
        - 如果你搜集的数据足够多，并且是按照上面的要求做到，那完全可以设置成1
        - 因为既然每个数据都是一场完整的战斗，那么真正的epochs其实是数据集的数量，除非你每场战斗是风格都不一样
    - **MODEL_WEIGHTS** 用以支持增量学习。默认为None。没有指定需要增量学习的模型的路径的话，就从零开始训练。
        - 当你搜集新的数据集之后想继续训练，可以把这个参数设置为要继续训练的模型的路径，这样就可以继续训练了
        - 当你在训练过程中发生了某种意外终止了训练，可以把这个参数设置为True，然后把发生意外时正在训练的那个数据集的序号设置为start，就可以继续训练了
- 如果想收集其他Boss的战斗数据，那就把target改成对应的Boss的名字（建议使用英文）

In [ ]:
from pysekiro.train_with_dl import train

target = 'Genichiro_Ashina' # 苇名弦一郎
train(
    target = target,
    start=2,
    end=102,
    batch_size=8,
    epochs=1
)

### 测试模型

In [ ]:
from learn_online import learn_online as test

test()

# 待更新